In [1]:
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns
import acquire as a
import prepare as p
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

 # Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [2]:
df = a.get_curriculum_logs_data()

In [3]:
df = p.prep_logs(df)

There are 3765 duplicate rows in your dataset - these will be dropped.
----------------


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 843565 entries, 2018-01-26 to 2021-04-21
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   time            843565 non-null  object        
 1   endpoint        843564 non-null  object        
 2   user_id         843565 non-null  int64         
 3   cohort_id       843565 non-null  float64       
 4   ip              843565 non-null  object        
 5   cohort          843565 non-null  object        
 6   slack           843565 non-null  object        
 7   start_date      843565 non-null  datetime64[ns]
 8   end_date        843565 non-null  datetime64[ns]
 9   created_at      843565 non-null  object        
 10  updated_at      843565 non-null  object        
 11  deleted_at      0 non-null       float64       
 12  program_id      843565 non-null  int64         
 13  accessed_after  843565 non-null  int64         
 14  program_name    8435

In [5]:
df_current = df.loc[df.accessed_after ==0]

In [6]:
df_current = df_current.loc[(df_current.index >= df_current.start_date) & (df_current.index <= df_current.end_date)]

In [7]:
user_df = pd.DataFrame(df_current.user_id.value_counts(dropna=False)).reset_index().\
                rename(index=str, columns={'index': 'user_id', 'user_id': 'user_count'})
user_df2 = pd.DataFrame(df_current.user_id.value_counts(dropna=False)/df_current.user_id.count()).reset_index().\
                rename(index=str, columns={'index': 'user_id', 'user_id': 'user_proba'})
user_df = user_df.merge(user_df2)

In [8]:
df_current = df_current.reset_index().merge(user_df, on=['user_id'], how='left').fillna(value=0).set_index('date')
df_current.user_proba.value_counts()

0.007405    4773
0.002214    4281
0.002709    3492
0.001753    3390
0.002600    3352
            ... 
0.000009       6
0.000008       5
0.000006       4
0.000002       4
0.000005       3
Name: user_proba, Length: 614, dtype: int64

In [9]:
df_current[df_current.user_count < 50]

,time,endpoint,user_id,cohort_id,ip,cohort,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,accessed_after,program_name,user_count,user_proba
date,,,,,,,,,,,,,,,,,
2018-01-26,11:24:17,appendix,24,21.0,97.105.19.61,Sequoia,#sequoia,2017-09-27,2018-02-15,2017-09-27 20:22:41,2017-09-27 20:22:41,0.0,2,0,Web Development,26,0.000040
2018-01-26,11:24:29,appendix/capstone-workbook,24,21.0,97.105.19.61,Sequoia,#sequoia,2017-09-27,2018-02-15,2017-09-27 20:22:41,2017-09-27 20:22:41,0.0,2,0,Web Development,26,0.000040
2018-01-26,13:58:07,spring/extra-features/file-upload,41,21.0,97.105.19.61,Sequoia,#sequoia,2017-09-27,2018-02-15,2017-09-27 20:22:41,2017-09-27 20:22:41,0.0,2,0,Web Development,28,0.000043
2018-01-26,16:53:53,spring,49,21.0,97.105.19.61,Sequoia,#sequoia,2017-09-27,2018-02-15,2017-09-27 20:22:41,2017-09-27 20:22:41,0.0,2,0,Web Development,25,0.000039
2018-01-26,16:53:54,spring/fundamentals/controllers,49,21.0,97.105.19.61,Sequoia,#sequoia,2017-09-27,2018-02-15,2017-09-27 20:22:41,2017-09-27 20:22:41,0.0,2,0,Web Development,25,0.000039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-21,10:25:09,/,976,139.0,72.26.31.59,Oberon,#oberon,2021-04-12,2021-10-01,2021-04-12 18:07:21,2021-04-12 18:07:21,0.0,2,0,Web Development,29,0.000045
2021-04-21,10:26:14,javascript-i/introduction/variables,973,139.0,70.123.219.174,Oberon,#oberon,2021-04-12,2021-10-01,2021-04-12 18:07:21,2021-04-12 18:07:21,0.0,2,0,Web Development,35,0.000054
2021-04-21,10:42:27,javascript-i/introduction/operators,972,139.0,97.117.115.55,Oberon,#oberon,2021-04-12,2021-10-01,2021-04-12 18:07:21,2021-04-12 18:07:21,0.0,2,0,Web Development,34,0.000053


In [10]:
hardly_working = df_current[df_current.user_count < 50]

In [13]:
hardly_working = hardly_working[['user_id','cohort','program_name','start_date','end_date', 'user_count']]

In [14]:
hardly_working = hardly_working.reset_index()

In [15]:
hardly_working = hardly_working.drop(columns=['date'])

In [16]:
hardly_working = hardly_working.drop_duplicates(subset='user_id', keep="first")
hardly_working

,user_id,cohort,program_name,start_date,end_date,user_count
0,24,Sequoia,Web Development,2017-09-27,2018-02-15,26
2,41,Sequoia,Web Development,2017-09-27,2018-02-15,28
3,49,Sequoia,Web Development,2017-09-27,2018-02-15,25
79,278,Voyageurs,Web Development,2018-05-29,2018-10-11,4
83,340,Zion,Web Development,2019-01-22,2019-06-04,18
101,371,Yosemite,Web Development,2018-11-05,2019-04-03,42
131,388,Andromeda,Web Development,2019-03-18,2019-07-30,8
151,487,Bayes,Data Science,2019-08-19,2020-01-30,17
168,529,Europa,Web Development,2019-11-04,2020-04-17,36
169,539,Europa,Web Development,2019-11-04,2020-04-17,5


In [17]:
hardly_working.program_name.value_counts()

Web Development    33
Data Science        3
Name: program_name, dtype: int64

In [18]:
hardly_working.cohort.value_counts()

Oberon       12
Neptune       4
Europa        3
Sequoia       3
Darden        2
Marco         2
Jupiter       2
Yosemite      1
Zion          1
Andromeda     1
Voyageurs     1
Bayes         1
Hyperion      1
Fortuna       1
Ganymede      1
Name: cohort, dtype: int64

In [19]:
hardly_working.start_date.value_counts()

2021-04-12    12
2021-03-15     4
2019-11-04     3
2017-09-27     3
2021-01-25     2
2020-07-13     2
2020-09-21     2
2019-01-22     1
2020-01-13     1
2020-03-23     1
2018-11-05     1
2019-08-19     1
2020-05-26     1
2018-05-29     1
2019-03-18     1
Name: start_date, dtype: int64

### I calculated hardly-accessed-curriculum students to be those who accessed it less than 50 times during their active class dates.

The only two programs who had people that met this criteria was Web Development, and Data Science.
What I'm noticing is:
 - majority of these students belong to web development cohorts
 - 3 cohorts in this list had not graduated by the time this dataset stopped collecting entries

# What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [20]:
df_grads = df.loc[df.accessed_after==1]

In [21]:
df_program_list = list(df.program_name.value_counts().index)

In [22]:
for program in df_program_list:     
        program_df = pd.DataFrame(df_grads[(df_grads.endpoint != '/')&(df_grads.endpoint != 'toc')&(df_grads.endpoint != 'search/search_index.json')&(df_grads.endpoint != 'index.html')].groupby('program_name')['endpoint'].value_counts())
        access_df = program_df.loc[[program]]
        access_df = access_df.endpoint.nlargest(3)
        #ds_df.append(cep_df)
        print(access_df)
        print('----------------------------------------------')
        print('\n')

program_name     endpoint    
Web Development  javascript-i    4207
                 spring          3740
                 html-css        3118
Name: endpoint, dtype: int64
----------------------------------------------


program_name  endpoint                                
Data Science  sql/mysql-overview                          275
              classification/overview                     266
              classification/scale_features_or_not.svg    211
Name: endpoint, dtype: int64
----------------------------------------------


program_name    endpoint    
Full Stack PHP  javascript-i    735
                html-css        540
                spring          496
Name: endpoint, dtype: int64
----------------------------------------------


program_name  endpoint                                   
Front End     content/html-css                               2
              content/html-css/gitbook/images/favicon.ico    1
              content/html-css/introduction.html            

# 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [23]:
df_program_list

['Web Development', 'Data Science', 'Full Stack PHP', 'Front End']

In [31]:
for program in df_program_list:     
        program_df = pd.DataFrame(df[(df.endpoint != '/')&(df.endpoint != 'toc')&(df.endpoint != 'search/search_index.json')&(df.endpoint != 'index.html')].groupby('program_name')['endpoint'].value_counts())
        access_df = program_df.loc[[program]]
        access_df = access_df.endpoint.nlargest(1)
        #ds_df.append(cep_df)
        print(access_df)
        print('\n')
        print('----------------------------------------------')
        print('\n')

program_name     endpoint    
Web Development  javascript-i    17392
Name: endpoint, dtype: int64


----------------------------------------------


program_name  endpoint               
Data Science  classification/overview    1783
Name: endpoint, dtype: int64


----------------------------------------------


program_name    endpoint    
Full Stack PHP  javascript-i    735
Name: endpoint, dtype: int64


----------------------------------------------


program_name  endpoint        
Front End     content/html-css    2
Name: endpoint, dtype: int64


----------------------------------------------




### 1a
-  Web Development: javascript-i
-  Data Science: classification
-  Full Stack PHP: javascript-1
-  Front End: html-css